# 🎯 Dynamic Position Sizer - Interactive Demo

This notebook walks you through all the features of the Dynamic Position Sizer tool.

**What you'll learn:**
1. How to analyze a single ticker
2. Understanding ATR (Average True Range)
3. Volatility regime classification
4. Trailing stop calculation
5. Position sizing based on risk
6. Comparing multiple tickers
7. Using mock data for testing

## 1. Setup & Imports

In [ ]:
# Install dependencies if needed (uncomment if required)
# !pip install yfinance pandas numpy

In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Import from the package
from main import analyze_ticker, analyze_watchlist
from config import Config, get_config
from data import YFinanceProvider, MockDataProvider
from indicators import compute_atr, compute_volatility_regime, compute_atr_multiple_periods
from position import StopRecommender, compute_trailing_stop, find_recent_high

print("✅ All imports successful!")

## 2. Quick Start - Analyze a Single Ticker

The simplest way to use the tool - just pass a ticker symbol!

In [ ]:
# Analyze NVDA with default settings
rec = analyze_ticker("NVDA")

print(f"📊 Analysis for {rec.ticker}")
print(f"{'='*40}")
print(f"Current Price:     ${rec.current_price:.2f}")
print(f"Suggested Stop:    ${rec.suggested_stop:.2f}")
print(f"Stop Distance:     {rec.stop_distance_pct:.1f}%")
print(f"Risk Per Share:    ${rec.risk_per_share:.2f}")
print(f"")
print(f"ATR (14-day):      ${rec.atr_14:.2f}")
print(f"Volatility Regime: {rec.volatility_regime.regime.upper()}")
print(f"Regime Multiplier: {rec.regime_adjusted_multiplier:.1f}x ATR")
print(f"")
print(f"Recent High:       ${rec.recent_high:.2f} ({rec.recent_high_date.strftime('%Y-%m-%d')})")

## 3. Position Sizing - How Many Shares for Your Risk?

The recommendation object includes helpers to calculate position size based on your risk tolerance.

In [ ]:
# Position sizing for different risk amounts
risk_amounts = [500, 1000, 2000, 5000]

print(f"📈 Position Sizing for {rec.ticker}")
print(f"Current Price: ${rec.current_price:.2f} | Stop: ${rec.suggested_stop:.2f}")
print(f"{'='*50}")
print(f"{'Risk Amount':<15} {'Shares':<10} {'Position Value':<15}")
print(f"{'-'*50}")

for risk in risk_amounts:
    shares = rec.shares_for_risk(risk)
    position_value = rec.position_value(risk)
    print(f"${risk:<14,} {shares:<10} ${position_value:,.2f}")

## 4. Analyze with Entry Price

If you have a specific entry price, the tool can calculate your initial stop and risk.

In [ ]:
# Analyze with a specific entry price
entry_price = 130.00  # Your entry price

rec_with_entry = analyze_ticker("NVDA", entry_price=entry_price)

print(f"📊 Analysis with Entry Price")
print(f"{'='*40}")
print(f"Entry Price:       ${rec_with_entry.entry_price:.2f}")
print(f"Current Price:     ${rec_with_entry.current_price:.2f}")
print(f"P&L per share:     ${rec_with_entry.current_price - entry_price:.2f}")
print(f"")
print(f"Initial Stop:      ${rec_with_entry.initial_stop:.2f}")
print(f"Current Stop:      ${rec_with_entry.suggested_stop:.2f}")
print(f"Stop moved up:     ${rec_with_entry.suggested_stop - rec_with_entry.initial_stop:.2f}")

## 5. Deep Dive - Understanding ATR Calculation

Let's look at how ATR (Average True Range) is calculated step by step.

In [ ]:
# Fetch raw data
provider = YFinanceProvider()
df = provider.get_ohlcv("NVDA", period="6mo")

print(f"📈 Raw OHLCV Data (last 10 days)")
print(df.tail(10))

In [ ]:
# Calculate ATR with different methods
atr_wilder = compute_atr(df, period=14, method="wilder")
atr_sma = compute_atr(df, period=14, method="sma")
atr_ema = compute_atr(df, period=14, method="ema")

print(f"📊 ATR Comparison (14-day period)")
print(f"{'='*40}")
print(f"Wilder's Smoothing: ${atr_wilder.current_atr:.2f}")
print(f"Simple MA:          ${atr_sma.current_atr:.2f}")
print(f"Exponential MA:     ${atr_ema.current_atr:.2f}")
print(f"")
print(f"Note: Wilder's is the default and most commonly used.")
print(f"      SMA is more responsive to recent changes.")
print(f"      EMA is a middle ground.")

In [ ]:
# Compare multiple ATR periods
atr_dict = compute_atr_multiple_periods(df, periods=[7, 14, 21])

print(f"📊 ATR Across Different Periods")
print(f"{'='*40}")
for period, result in atr_dict.items():
    print(f"ATR({period}): ${result.current_atr:.2f}")

print(f"")
print(f"Shorter periods (7) = More responsive to recent volatility")
print(f"Longer periods (21) = Smoother, less reactive")

## 6. Volatility Regime Analysis

The tool classifies current volatility into regimes by comparing to historical data.

In [ ]:
# Compute volatility regime
regime = compute_volatility_regime(df, atr_period=14, lookback_days=252)

print(f"📊 Volatility Regime Analysis")
print(f"{'='*50}")
print(f"Current ATR:       ${regime.current_atr:.2f}")
print(f"Historical Mean:   ${regime.historical_mean:.2f}")
print(f"Historical Std:    ${regime.historical_std:.2f}")
print(f"")
print(f"Percentile:        {regime.percentile:.1f}%")
print(f"Z-Score:           {regime.z_score:.2f}")
print(f"")
print(f"🎯 Regime: {regime.regime.upper()}")
print(f"")
print(f"Regime Guide:")
print(f"  LOW (0-25%):      Use 1.5x ATR multiplier (tight stops)")
print(f"  NORMAL (25-75%):  Use 2.0x ATR multiplier (standard)")
print(f"  ELEVATED (75-90%): Use 2.5x ATR multiplier (wider stops)")
print(f"  EXTREME (90%+):   Use 3.0x ATR multiplier (very wide stops)")

## 7. Trailing Stop Calculation - Step by Step

In [ ]:
# Find the recent high
recent_high, high_date = find_recent_high(df, lookback_days=20)

print(f"📈 Recent High Analysis")
print(f"{'='*40}")
print(f"Recent High:  ${recent_high:.2f}")
print(f"Date:         {high_date.strftime('%Y-%m-%d')}")
print(f"Current:      ${df['Close'].iloc[-1]:.2f}")
print(f"From High:    {((df['Close'].iloc[-1] / recent_high) - 1) * 100:.1f}%")

In [ ]:
# Calculate trailing stop manually
current_price = df['Close'].iloc[-1]
atr = atr_wilder.current_atr
multiplier = 2.0  # Base multiplier

# Apply regime adjustment
regime_multipliers = {
    "low": 1.5,
    "normal": 2.0,
    "elevated": 2.5,
    "extreme": 3.0
}
adjusted_multiplier = regime_multipliers[regime.regime]

# Calculate stop
stop_distance = adjusted_multiplier * atr
trailing_stop = recent_high - stop_distance

print(f"📊 Trailing Stop Calculation")
print(f"{'='*50}")
print(f"")
print(f"Formula: Stop = Recent High - (Multiplier × ATR)")
print(f"")
print(f"Recent High:        ${recent_high:.2f}")
print(f"ATR (14):           ${atr:.2f}")
print(f"Base Multiplier:    {multiplier}x")
print(f"Regime:             {regime.regime}")
print(f"Adjusted Multiplier: {adjusted_multiplier}x")
print(f"")
print(f"Stop Distance:      ${stop_distance:.2f} ({adjusted_multiplier} × {atr:.2f})")
print(f"")
print(f"🎯 Trailing Stop:   ${trailing_stop:.2f}")
print(f"   Risk per share:  ${current_price - trailing_stop:.2f}")
print(f"   Stop Distance:   {((current_price - trailing_stop) / current_price) * 100:.1f}%")

## 8. Compare Multiple Tickers

Analyze your entire watchlist at once!

In [ ]:
# Analyze multiple tickers
tickers = ["NVDA", "AAPL", "TSLA", "AMD", "META"]
recommendations = analyze_watchlist(tickers)

print(f"📊 Watchlist Analysis")
print(f"{'='*80}")
print(f"{'Ticker':<8} {'Price':<10} {'Stop':<10} {'Risk %':<10} {'ATR':<10} {'Regime':<12}")
print(f"{'-'*80}")

for rec in recommendations:
    print(f"{rec.ticker:<8} ${rec.current_price:<9.2f} ${rec.suggested_stop:<9.2f} {rec.stop_distance_pct:<9.1f}% ${rec.atr_14:<9.2f} {rec.volatility_regime.regime.upper():<12}")

In [ ]:
# Create a summary DataFrame
summary_data = []
for rec in recommendations:
    summary_data.append({
        "Ticker": rec.ticker,
        "Price": rec.current_price,
        "Stop": rec.suggested_stop,
        "Risk %": rec.stop_distance_pct,
        "ATR(14)": rec.atr_14,
        "Regime": rec.volatility_regime.regime,
        "Multiplier": rec.regime_adjusted_multiplier,
        "Shares ($1k risk)": rec.shares_for_risk(1000)
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values("Risk %", ascending=False)

print(f"\n📈 Summary Table (sorted by risk %)")
summary_df

## 9. Using Mock Data for Testing

Test the tool without hitting the network using synthetic data.

In [ ]:
# Create a mock data provider
mock_provider = MockDataProvider(seed=42)  # Seed for reproducibility

# Create recommender with mock data
mock_recommender = StopRecommender(data_provider=mock_provider)

# Analyze using mock data
mock_rec = mock_recommender.analyze("NVDA")

print(f"📊 Mock Data Analysis for {mock_rec.ticker}")
print(f"{'='*40}")
print(f"(Using synthetic data - great for testing!)")
print(f"")
print(f"Current Price:     ${mock_rec.current_price:.2f}")
print(f"Suggested Stop:    ${mock_rec.suggested_stop:.2f}")
print(f"ATR (14):          ${mock_rec.atr_14:.2f}")
print(f"Volatility Regime: {mock_rec.volatility_regime.regime.upper()}")

In [ ]:
# Look at the mock data
mock_df = mock_provider.get_ohlcv("TSLA", period="3mo")

print(f"📈 Mock OHLCV Data for TSLA")
print(f"(Generated using Geometric Brownian Motion with volatility clustering)")
print(f"")
print(mock_df.tail(10))

## 10. Custom Configuration

Customize the analysis parameters to fit your trading style.

In [ ]:
# View default configuration
config = get_config()

print(f"📋 Default Configuration")
print(f"{'='*40}")
print(f"")
print(f"ATR Settings:")
print(f"  Period:  {config.atr.default_period}")
print(f"  Method:  {config.atr.method}")
print(f"")
print(f"Regime Thresholds (percentiles):")
print(f"  Low:      0-{config.volatility_regime.thresholds['low']}%")
print(f"  Normal:   {config.volatility_regime.thresholds['low']}-{config.volatility_regime.thresholds['normal']}%")
print(f"  Elevated: {config.volatility_regime.thresholds['normal']}-{config.volatility_regime.thresholds['elevated']}%")
print(f"  Extreme:  {config.volatility_regime.thresholds['elevated']}%+")
print(f"")
print(f"Regime Multipliers:")
for regime, mult in config.trailing_stop.regime_adjustments.items():
    print(f"  {regime.capitalize()}: {mult}x ATR")

In [ ]:
# Analyze with custom parameters
custom_rec = analyze_ticker(
    "AAPL",
    atr_period=21,           # Longer ATR period (smoother)
    base_multiplier=2.5,     # Wider base multiplier
    use_regime_adjustment=False  # Disable regime adjustment
)

print(f"📊 Custom Analysis for {custom_rec.ticker}")
print(f"{'='*40}")
print(f"ATR Period:        21 (custom)")
print(f"Base Multiplier:   2.5x (custom)")
print(f"Regime Adjustment: Disabled")
print(f"")
print(f"Current Price:     ${custom_rec.current_price:.2f}")
print(f"Suggested Stop:    ${custom_rec.suggested_stop:.2f}")
print(f"Stop Distance:     {custom_rec.stop_distance_pct:.1f}%")

## 11. Visualizing ATR Over Time

Let's plot the ATR and price to see how volatility changes.

In [ ]:
# This cell requires matplotlib - uncomment to install
# !pip install matplotlib

try:
    import matplotlib.pyplot as plt
    
    # Fetch data
    provider = YFinanceProvider()
    df = provider.get_ohlcv("NVDA", period="1y")
    
    # Calculate ATR
    atr_result = compute_atr(df, period=14, method="wilder")
    
    # Create figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot price
    ax1.plot(df.index, df['Close'], label='Close Price', color='blue')
    ax1.set_ylabel('Price ($)')
    ax1.set_title('NVDA Price and ATR(14)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Plot ATR
    ax2.plot(atr_result.atr_series.index, atr_result.atr_series, label='ATR(14)', color='orange')
    ax2.axhline(y=atr_result.atr_series.mean(), color='red', linestyle='--', label=f'Mean ATR: ${atr_result.atr_series.mean():.2f}')
    ax2.set_ylabel('ATR ($)')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("matplotlib not installed. Run: pip install matplotlib")
    print("Skipping visualization...")

## 12. Full Recommendation Object Exploration

Let's see everything that's available in the StopRecommendation object.

In [ ]:
# Get a full recommendation
rec = analyze_ticker("TSLA", entry_price=250.0)

print(f"📋 Full StopRecommendation Object")
print(f"{'='*50}")
print(f"")
print(f"Basic Info:")
print(f"  ticker:              {rec.ticker}")
print(f"  generated_at:        {rec.generated_at}")
print(f"  data_period:         {rec.data_period}")
print(f"")
print(f"Prices:")
print(f"  current_price:       ${rec.current_price:.2f}")
print(f"  entry_price:         ${rec.entry_price:.2f}")
print(f"  recent_high:         ${rec.recent_high:.2f}")
print(f"  recent_high_date:    {rec.recent_high_date}")
print(f"")
print(f"Stops:")
print(f"  suggested_stop:      ${rec.suggested_stop:.2f}")
print(f"  initial_stop:        ${rec.initial_stop:.2f}")
print(f"  stop_distance_pct:   {rec.stop_distance_pct:.1f}%")
print(f"  risk_per_share:      ${rec.risk_per_share:.2f}")
print(f"")
print(f"ATR Values:")
print(f"  atr_7:               ${rec.atr_7:.2f}")
print(f"  atr_14:              ${rec.atr_14:.2f}")
print(f"  atr_21:              ${rec.atr_21:.2f}")
print(f"")
print(f"Multipliers:")
print(f"  base_multiplier:     {rec.base_multiplier}x")
print(f"  regime_adjusted:     {rec.regime_adjusted_multiplier}x")
print(f"")
print(f"Volatility Regime:")
print(f"  regime:              {rec.volatility_regime.regime}")
print(f"  percentile:          {rec.volatility_regime.percentile:.1f}%")
print(f"  z_score:             {rec.volatility_regime.z_score:.2f}")
print(f"  historical_mean:     ${rec.volatility_regime.historical_mean:.2f}")
print(f"  historical_std:      ${rec.volatility_regime.historical_std:.2f}")

## 🎉 You're Ready!

You now know how to:
- Analyze single tickers and watchlists
- Understand ATR and volatility regimes
- Calculate trailing stops
- Size positions based on risk
- Use mock data for testing
- Customize parameters

**Next steps:**
- Try analyzing your own watchlist
- Experiment with different ATR periods and multipliers
- Use the CLI for quick lookups: `python cli.py analyze TICKER`